In [ ]:
import torch
import numpy as np
import torch.nn as nn

In [ ]:
!wget 'https://raw.githubusercontent.com/r0ckYr/GenerativeAI/main/text.txt'

--2023-08-09 11:34:55--  https://raw.githubusercontent.com/r0ckYr/GenerativeAI/main/text.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115393 (1.1M) [text/plain]
Saving to: ‘text.txt’

text.txt            100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2023-08-09 11:34:56 (20.1 MB/s) - ‘text.txt’ saved [1115393/1115393]



In [ ]:
!ls

sample_data  text.txt


In [ ]:
with open('text.txt', 'r') as f:
  input = f.read()

In [ ]:
input = input[:10000]

In [ ]:
input[100:164]

' are all resolved rather to die than to famish?\n\nAll:\nResolved. '

In [ ]:
chars = sorted(list(set(input)))

In [ ]:
chars[:10]

['\n', ' ', '!', "'", ',', '-', '.', ':', ';', '?']

In [ ]:
stoi = {}
itos = {}
i = 0
for ch in chars:
  stoi[ch] = i
  itos[i] = ch
  i = i+1


In [ ]:
vectored_input = []
for ch in input:
  vectored_input.append(stoi[ch])

In [ ]:
vectored_input[:10]

[15, 39, 48, 49, 50, 1, 12, 39, 50, 39]

In [ ]:
print(vectored_input[:10])
print(input[:10])

[15, 39, 48, 49, 50, 1, 12, 39, 50, 39]
First Citi


In [ ]:
batch_size = 32
block_size = 128

In [ ]:
#do this for multiple block sizes
X = []
Y = []
for i in range(0, len(vectored_input) - block_size-block_size, 1):
    X.append(vectored_input[i:i + block_size])
    Y.append(vectored_input[i + block_size:i+block_size+block_size])

In [ ]:
len(Y)

9744

In [ ]:
for i in range(10):
  print(X[i]," : ", Y[i])

[15, 39, 48, 49, 50, 1, 12, 39, 50, 39, 56, 35, 44, 7, 0, 11, 35, 36, 45, 48, 35, 1, 53, 35, 1, 46, 48, 45, 33, 35, 35, 34, 1, 31, 44, 55, 1, 36, 51, 48, 50, 38, 35, 48, 4, 1, 38, 35, 31, 48, 1, 43, 35, 1, 49, 46, 35, 31, 41, 6, 0, 0, 10, 42, 42, 7, 0, 25, 46, 35, 31, 41, 4, 1, 49, 46, 35, 31, 41, 6, 0, 0, 15, 39, 48, 49, 50, 1, 12, 39, 50, 39, 56, 35, 44, 7, 0, 30, 45, 51, 1, 31, 48, 35, 1, 31, 42, 42, 1, 48, 35, 49, 45, 42, 52, 35, 34, 1, 48, 31, 50, 38, 35, 48, 1, 50, 45, 1]  :  [34, 39, 35, 1, 50, 38, 31, 44, 1, 50, 45, 1, 36, 31, 43, 39, 49, 38, 9, 0, 0, 10, 42, 42, 7, 0, 24, 35, 49, 45, 42, 52, 35, 34, 6, 1, 48, 35, 49, 45, 42, 52, 35, 34, 6, 0, 0, 15, 39, 48, 49, 50, 1, 12, 39, 50, 39, 56, 35, 44, 7, 0, 15, 39, 48, 49, 50, 4, 1, 55, 45, 51, 1, 41, 44, 45, 53, 1, 12, 31, 39, 51, 49, 1, 20, 31, 48, 33, 39, 51, 49, 1, 39, 49, 1, 33, 38, 39, 35, 36, 1, 35, 44, 35, 43, 55, 1, 50, 45, 1, 50, 38, 35, 1, 46, 35, 45, 46, 42, 35, 6, 0, 0, 10, 42, 42, 7, 0]
[39, 48, 49, 50, 1, 12, 39, 50, 

In [ ]:
Y[-2]

[44,
 39,
 51,
 49,
 1,
 11,
 48,
 51,
 50,
 51,
 49,
 4,
 0,
 25,
 39,
 33,
 39,
 44,
 39,
 51,
 49,
 1,
 28,
 35,
 42,
 51,
 50,
 51,
 49,
 4,
 1,
 31,
 44,
 34,
 1,
 17,
 1,
 41,
 44,
 45,
 53,
 1,
 44,
 45,
 50,
 5,
 5,
 3,
 25,
 34,
 35,
 31,
 50,
 38,
 2,
 0,
 26,
 38,
 35,
 1,
 48,
 31,
 32,
 32,
 42,
 35,
 1,
 49,
 38,
 45,
 51,
 42,
 34,
 1,
 38,
 31,
 52,
 35,
 1,
 36,
 39,
 48,
 49,
 50,
 1,
 51,
 44,
 48,
 45,
 45,
 36,
 3,
 34,
 1,
 50,
 38,
 35,
 1,
 33,
 39,
 50,
 55,
 4,
 0,
 14,
 48,
 35,
 1,
 49,
 45,
 1,
 46,
 48,
 35,
 52,
 31,
 39,
 42,
 3,
 34,
 1,
 53,
 39,
 50,
 38,
 1,
 43,
 35]

In [ ]:
X = torch.tensor(X, dtype=torch.long)
Y = torch.tensor(Y, dtype=torch.long)

In [ ]:
for i in range(10):
  print(X[i]," : ", Y[i])

tensor([15, 39, 48, 49, 50,  1, 12, 39, 50, 39, 56, 35, 44,  7,  0, 11, 35, 36,
        45, 48, 35,  1, 53, 35,  1, 46, 48, 45, 33, 35, 35, 34,  1, 31, 44, 55,
         1, 36, 51, 48, 50, 38, 35, 48,  4,  1, 38, 35, 31, 48,  1, 43, 35,  1,
        49, 46, 35, 31, 41,  6,  0,  0, 10, 42, 42,  7,  0, 25, 46, 35, 31, 41,
         4,  1, 49, 46, 35, 31, 41,  6,  0,  0, 15, 39, 48, 49, 50,  1, 12, 39,
        50, 39, 56, 35, 44,  7,  0, 30, 45, 51,  1, 31, 48, 35,  1, 31, 42, 42,
         1, 48, 35, 49, 45, 42, 52, 35, 34,  1, 48, 31, 50, 38, 35, 48,  1, 50,
        45,  1])  :  tensor([34, 39, 35,  1, 50, 38, 31, 44,  1, 50, 45,  1, 36, 31, 43, 39, 49, 38,
         9,  0,  0, 10, 42, 42,  7,  0, 24, 35, 49, 45, 42, 52, 35, 34,  6,  1,
        48, 35, 49, 45, 42, 52, 35, 34,  6,  0,  0, 15, 39, 48, 49, 50,  1, 12,
        39, 50, 39, 56, 35, 44,  7,  0, 15, 39, 48, 49, 50,  4,  1, 55, 45, 51,
         1, 41, 44, 45, 53,  1, 12, 31, 39, 51, 49,  1, 20, 31, 48, 33, 39, 51,
        49,  1, 39,

In [ ]:
class ShakespeareModel(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers):
        super(ShakespeareModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden):
        x = self.embedding(x)
        x, hidden = self.lstm(x, hidden)
        x = self.fc(x)
        return x, hidden

In [ ]:
# Define hyperparameters
input_size = block_size
hidden_size = 128
num_layers = 2
output_size = len(chars)
learning_rate = 0.001
num_epochs = 1
vocab_size = len(chars)
hidden = None

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = ShakespeareModel(vocab_size, hidden_size, num_layers).to(device)

In [ ]:
X = X.to(device)
Y = Y.to(device)

In [ ]:
model.train()

ShakespeareModel(
  (embedding): Embedding(57, 128)
  (lstm): LSTM(128, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=57, bias=True)
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    total_loss = 0
    for inputs, lables in zip(X,Y):
        inputs = inputs.to(device)
        labels = lables.to(device)
        # Convert inputs to a tensor
        input_tensor = torch.tensor(inputs, dtype=torch.long)

        # Forward pass
        outputs, hidden = model(input_tensor, None)
        output_tensor = outputs  # Extract the output tensor from the tuple

        # Reshape outputs for loss calculation
        output_reshaped = output_tensor.view(-1, len(chars))

        # Calculate the loss using the reshaped outputs and corresponding targets
        loss = criterion(output_reshaped, lables.view(-1))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss = total_loss + loss.item()
    print(f'Finished epoch {epoch+1}, latest loss {loss.item():.4f}, Loss: {total_loss:.4f}')

<ipython-input-35-cfa742528a41>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tensor = torch.tensor(inputs, dtype=torch.long)


KeyboardInterrupt: ignored

In [ ]:
model.eval()

In [ ]:
import numpy as np


def generate_text(model, initial_text, stoi, itos, length, temperature=1.0):

    with torch.no_grad():
        input_text = list(initial_text)
        input_tensor = torch.tensor([stoi[ch] for ch in input_text], dtype=torch.long).unsqueeze(0)
        hidden = None

        generated_text = input_text.copy()
        for _ in range(length):
          output, hidden = model(input_tensor, hidden)
          output_probs = torch.softmax(output[:, -1, :] / temperature, dim=-1)

          # Sample the next character using the output probabilities
          next_char_idx = torch.multinomial(output_probs, num_samples=1).squeeze().item()
          next_char = itos[next_char_idx]

          generated_text.append(next_char)
          input_tensor = torch.tensor([[next_char_idx]], dtype=torch.long)

        return ''.join(generated_text)


In [ ]:
initial_text = "iii"
generated_length = 500
temperature = 0.001

generated_text = generate_text(model, initial_text, stoi, itos, generated_length, temperature)
print(generated_text)